In [5]:
from Sindonews import Sindonews
import mysql.connector

In [6]:
obj = Sindonews()

In [ ]:
links = obj.getAllBerita([], 1, 0, 0, '2018-08-01')

page  1
https://index.sindonews.com/index/0/0?t=2018-08-01
Insert berita  Batal Arsiteki Aston Villa, Henry Siap Tangani Mohamed Salah dkk
masuk
Insert berita  Perkuat Kerja Sama, Menteri PPPA Temui Pejabat Tinggi Iran
masuk
Insert berita  Ratusan Kendaraan Roda Empat di Mimika Terjaring Razia
masuk
Insert berita  Trump Serukan Jaksa Agung AS Hentikan Penyelidikan Soal Rusia
masuk
Insert berita  KPK Rekrut Ulang Direktur Penyidikan Baru
masuk
Insert berita  Toyota Vios SE Resmi Mengaspal di Vietnam
masuk
Insert berita  Chandra Asri Raih Kredit Modal Kerja USD120 Juta
masuk
Insert berita  AS: Jasad yang Dikembalikan Korut Kemungkinan Tentara Amerika
masuk
Insert berita  Kemendagri Pastikan Pelayanan Publik di Bekasi Berjalan Normal
masuk
Insert berita  Klasemen dan Jadwal Pertandingan Piala AFF U-16, Kamis (2/8/2018)
masuk
page  2
https://index.sindonews.com/index/0/10?t=2018-08-01
Insert berita  Sebanyak 139 TKI Bermasalah Dideportasi dari Malaysia
masuk
Insert berita  Lewat Surat, Ara

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from requests.exceptions import ConnectionError
import unicodedata
import mysql.connector

In [ ]:
articles = {}

In [ ]:
url = "https://index.sindonews.com/index/0/20?t=2018-08-04"

In [ ]:
response = requests.get(url)
# Extract HTML texts contained in Response object: html
html = response.text
# Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [ ]:
el_page = soup.find('div', class_="pagination")
el_page
# if el_page:
max_page = el_page.findAll('a')[-1]['data-ci-pagination-page']
# max_page
active_page = el_page.find('li', class_="active").get_text(strip=True)
active_page
